# Imports

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Widgets

In [0]:
dbutils.widgets.removeAll()

In [0]:
dbutils.widgets.text("storageName", "adlsprojectsmartdata")
dbutils.widgets.text("container", "raw")
dbutils.widgets.text("catalog", "project_smartdata")
dbutils.widgets.text("schema", "bronze")

# Constants

In [0]:
storage_name = dbutils.widgets.get("storageName")
container = dbutils.widgets.get("container")
catalog =  dbutils.widgets.get("catalog")
schema =  dbutils.widgets.get("schema")

# Paths

In [0]:
path_base = f"abfss://{container}@{storage_name}.dfs.core.windows.net/"
path_warranty =  f"{path_base}warranty.csv"

# Structure

In [0]:
warranty_schema =  StructType(fields=[
    StructField("claim_id", StringType(), False),
    StructField("claim_date", StringType(), True),
    StructField("sale_id", StringType(), True),
    StructField("repair_status", StringType(), False)
])

# Read source

In [0]:
df_warranty =  spark.read.option("header", True)\
                      .schema(warranty_schema)\
                      .csv(path_warranty)

In [0]:
df_warranty_final =  df_warranty.withColumn("processed_at", current_timestamp())

# Save

In [0]:
df_warranty_final.write.mode("overwrite").saveAsTable(f"{catalog}.{schema}.warranty")